In [148]:
%config IPCompleter.greedy=True

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io
from scipy import optimize
from random import sample
import random

In [8]:
data = scipy.io.loadmat("ex4data1.mat")
weights = scipy.io.loadmat("ex4weights.mat")

In [14]:
X = data['X']; y = data['y']
X = np.c_[np.ones(X.shape[0]), X]
theta = [weights['Theta1'], weights['Theta2']]

In [15]:
print("X", X.shape)
print("y", y.shape)
print("theta1", weights['Theta1'].shape)
print("theta2", weights['Theta2'].shape)

X (5000, 401)
y (5000, 1)
theta1 (25, 401)
theta2 (10, 26)


In [16]:
def displayData(X, index_list=[]):
    row, col = 10, 10
    h, w = 20, 20
    if len(index_list) == 0:
        index_list = sample(range(X.shape[0]), row * col)
        
    picture = np.zeros((row*h, col*w))
    print(picture.shape)
    
    i, j = 0, 0
    for index in index_list:
        if j == col:
            j = 0
            i += 1
        row_position = i * h
        col_position = j * w
        image = X[index].reshape(h, w).T
        #print(image.shape)
        #print(row_position, row_position + h, col_position, col_position + w)
        picture[row_position:row_position + h, col_position:col_position + w] = image
        j += 1
    
    plt.imshow(picture, cmap='gray')
    
displayData(X)

(200, 200)


ValueError: cannot reshape array of size 401 into shape (20,20)

In [143]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

def sigmoidGradient(z):
    s = sigmoid(z)
    return s*(1-s)

def feedforward(X, theta):
    a = X
    depth = len(theta)
    
    for i in range(depth):
        w = theta[i]
        print(a.shape, w.T.shape)
        z = a.dot(w.T)
        a = sigmoid(z)
        print(a.shape)
        if i != depth -1:
            a = np.c_[np.ones(a.shape[0]), a]
    return a

In [144]:
def costFunction(theta, X, y, alpha=1.0):
    y_table = np.eye(10)
    h = feedforward(X, theta)

    total_cost = 0.0
    m = h.shape[0]
    
    for i in range(m):
        left = -y_table[y[i]-1].dot(np.log(h[i]))
        right = (1-y_table[y[i]-1]).dot(np.log(1-h[i]))
        total_cost += (left - right)/m
    print("total_cost:", total_cost)
    
    reg_term = 0.0
    for i in range(len(theta)):
        reg_term += np.sum(theta[i]*theta[i])        
    reg_term *= alpha / (2*m)
    print("reg_term:", reg_term)
    total_cost += reg_term
    return total_cost
   
print(costFunction(theta, X, y))

(5000, 401) (401, 25)
(5000, 25)
(5000, 26) (26, 10)
(5000, 10)
total_cost: [0.28762917]
reg_term: 0.09685863108157507
[0.3844878]


In [145]:
print(sigmoidGradient(0))

0.25


In [147]:
temp = np.array([[0,0,0], [0,0,0]])
print(sigmoidGradient(temp))

[[0.25 0.25 0.25]
 [0.25 0.25 0.25]]


In [166]:
initial_theta = []
epsilon = 0.12
for i in range(len(theta)):
    row, col = theta[i].shape[0], theta[i].shape[1]
    initial_theta.append(np.random.uniform(-epsilon, epsilon, row * col).reshape(row, col))